Please make sure that the data files are located in the folder "season_1/training_data/order_data/". Then run all the cells.

In [1]:
import pandas as pd
from os import walk

"""import the order datas into a dataframe""" 

# Use path to data files.
order_path = "season_2/training_data/order_data/" 
_, _, filenames = next(walk(order_path), (None, None, []))

mSeq = len(filenames)
frame = pd.DataFrame()
list_ = []

# Uncomment these lines to import all the data 
for i in range(mSeq-1):
    # White Space as Delim, import all the data file.
    df = pd.read_csv(str(order_path) + filenames[i+1],  delim_whitespace = 1, dtype = str, header = None) 
    list_.append(df)
orders = pd.concat(list_)

# Only import the data of the first day
# orders = pd.read_csv(str(order_path) + filenames[1], delim_whitespace = 1, dtype = str,  header = None) 

# Rename all the columns.
orders.columns = ['order_id', 'driver_id', 'passenger_id', 
                  'start_district_hash', 'dest_district_hash', 
                  'price', 'date', 'time']
# Convert the Date and Time column in orders from an object to a datetime value. 
# This makes it easier to work with as a date.
# Extract the year and month from from date_time, and assign them to their own columns.
orders['datetime'] = pd.to_datetime(orders['date'] + ' ' + orders['time'])
orders = orders.drop('date', 1)
orders = orders.drop('time', 1)
orders['day'] = orders["datetime"].dt.day
# Every 10 minutes is considered as a time slot. It can be calculated by following fomula
orders['time_slot'] = orders["datetime"].dt.hour*6 + orders["datetime"].dt.minute//10

In [2]:
"""Categorize the start and destination with its id in """

# Import the hash id into a dictionary
cluster_path = "season_1/training_data/cluster_map/"
cluster = {}
with open(cluster_path + "cluster_map") as f:
    for line in f:
       (key, val) = line.split()
       cluster[str(key)] = val
        
# Update the start and destination district ID
orders['start_district_id'] = orders['start_district_hash'].map(cluster)
orders['dest_district_id'] = orders['dest_district_hash'].map(cluster)

In [3]:
# Take a look at the data. The number of the data collected in each district
# is not evenly distributed. District 51 can be considered as a high occupancied
# zone. We're more likely to dig into this kind of areas.
# Also, interstingly there are more orders in from 12am to 1am, which is not expected.
# Probably because of in this period there are still demands but not enough taxis or 
# other public transportation.

print(orders["start_district_id"].value_counts())
print(orders["time_slot"].value_counts())

51    1328539
23     772902
8      745674
48     542984
28     493144
1      444405
7      423113
37     396614
46     337205
12     277499
20     273773
24     230680
14     228234
21     196781
42     186923
27     166409
4      141162
19     130188
22     101472
26     100346
25      86020
9       80919
29      63220
11      58683
36      57057
2       51832
6       45003
54      40992
35      35024
57      33598
       ...   
39      23536
47      23200
41      21286
16      21286
34      20383
18      19876
33      18385
40      16617
17      14444
56      11870
45      11208
58       8574
5        8192
10       8089
3        7803
59       6804
49       6719
32       6523
65       6183
30       6156
61       5387
43       5086
55       4242
50       4208
52       3257
44       3232
60       3119
62       1597
15       1291
63       1244
Name: start_district_id, dtype: int64
51     142540
49     142397
105    137069
48     136041
52     135507
50     134670
108    124015
106    123

In [4]:
# Now try to generate the gap data. We're looking for the orders which were not answered by a driver.
# Hence we extract the orders with missing driver and count the number of them in each distrct and time slot.
# The number of the not answered orders should be the gap we're looking for

gaporders = orders[pd.isnull(orders['driver_id'])]
gap = pd.DataFrame({'gap' : gaporders.groupby( ['day', 'start_district_id', 'time_slot'] ).size()}).reset_index()
gap['key'] = gap.day.astype(str).str.cat(gap.start_district_id.astype(str), sep='_').str.cat(gap.time_slot.astype(str), sep='_')
gap["district_id"] = gap['start_district_id']

In [5]:
# Create a new DataFrame to save the useful information
time_slot = pd.DataFrame({'stat' : orders.groupby(["time_slot"]).size()})
szone = pd.DataFrame({'stat' : orders.groupby(["start_district_id"]).size()})
ezone = pd.DataFrame({'stat' : orders.groupby(["dest_district_id"]).size()})
day = pd.DataFrame({'stat' : orders.groupby(["day"]).size()})
print (len(time_slot),len(szone), len(ezone), len(day))
in_d = pd.DataFrame(columns=['day', 'start_district_id', 'time_slot',"gap"])
n = 1
for d in day.index.values:    
    date = d
    print('day: '+ str(d))
    for z in szone.index.values:
        start = z
        for t in time_slot.index.values:
            time = t
            in_d.loc[n]=[date, start, time, 0.0]
            n+=1

144 66 66 21
day: 1
day: 2
day: 3
day: 4
day: 5
day: 6
day: 7
day: 8
day: 9
day: 10
day: 11
day: 12
day: 13
day: 14
day: 15
day: 16
day: 17
day: 18
day: 19
day: 20
day: 21


In [ ]:
in_d.to_csv('in_d.txt', sep = ' ')

In [ ]:
in_da = pd.read_csv('in_d.txt', delim_whitespace = 1, dtype = float) 

In [36]:
# Create a new column called key which combines day, start district id and time slot. 
in_d['key'] = in_d["day"]
in_d['key'] = in_d.day.astype(int).astype(str).str.cat(in_d.start_district_id.astype(str), sep='_').str.cat(in_d.time_slot.astype(int).astype(str), sep='_')
in_d["district_id"] = in_d["start_district_id"]

In [37]:
# Generate demand, supply and future supply (which depends on the destination of each order). Our naive model 
# will be based on these variables. Future supply will be an interesting attribute to dig in,

demand = pd.DataFrame({'demand' : orders.groupby( ['day', 'start_district_id', 'time_slot'] ).size()}).reset_index()
supply = pd.DataFrame({'demand' : orders.groupby( ['day', 'driver_id', 'time_slot'] ).size()}).reset_index()
fsupply = pd.DataFrame({'future_supply' : orders.groupby( ['day', 'dest_district_id', 'time_slot'] ).size()}).reset_index()
demand["district_id"] = demand['start_district_id']
fsupply["district_id"] = fsupply['dest_district_id']

In [38]:
# fill in the table with the information generated.

def join_value(left, right, key):
    left['key'] = left.day.astype(int).astype(str).str.cat(left.district_id.astype(int).astype(str), sep='_').str.cat(left.time_slot.astype(int).astype(str), sep='_')
    right['key'] = right.day.astype(int).astype(str).str.cat(right.district_id.astype(int).astype(str), sep='_').str.cat(right.time_slot.astype(int).astype(str), sep='_')
    left_a = left.set_index(key)
    right_a = right.set_index(key)

    res = left_a.loc[:, left_a.columns.union(right_a.columns)]
    res.update(right_a)
    return res

res = join_value(in_d, gap, "key")
res = join_value(res, demand, "key")
res = join_value(res, fsupply, "key")
res["time"] = res["time_slot"]/144*24
res.drop(["dest_district_id", "start_district_id"], axis=1,inplace=True) 
res["supply"] = res["demand"]-res["gap"]

print (res.sort_values(["gap"], ascending=False))


            day  demand district_id  future_supply     gap  time_slot  \
key                                                                     
1_51_5      1.0  4362.0          51         1119.0  3872.0        5.0   
1_51_4      1.0  4296.0          51         1131.0  3784.0        4.0   
1_51_6      1.0  4056.0          51          984.0  3602.0        6.0   
1_51_7      1.0  3988.0          51          866.0  3514.0        7.0   
1_51_8      1.0  3611.0          51          826.0  3149.0        8.0   
1_51_3      1.0  3620.0          51         1018.0  3096.0        3.0   
1_51_9      1.0  3367.0          51          717.0  2947.0        9.0   
1_51_10     1.0  2821.0          51          586.0  2392.0       10.0   
1_51_11     1.0  2224.0          51          473.0  1839.0       11.0   
1_51_2      1.0  2363.0          51          741.0  1698.0        2.0   
1_51_12     1.0  1859.0          51          411.0  1603.0       12.0   
1_51_13     1.0  1669.0          51          434.0 

In [39]:
res['fri'] = res['day']==1
res['sat'] = res['day']==2
res['sun'] = res['day']==3
res['mon'] = res['day']==4
res['tue'] = res['day']==5
res['wed'] = res['day']==6
res['thu'] = res['day']==7

res['gap_slot'] = res['gap']
res['gap_max'] = 100
res['gap_slot'] = res[['gap_slot', 'gap_max']].min(axis=1)
res['gap_slot'] = res['gap_slot']//5

In [40]:
input = pd.DataFrame(res[['day', 'district_id', 'gap', 'time_slot']])
output = pd.DataFrame(res[['day', 'time_slot', 'gap', 'gap_slot']])

output = output.ix[1:]
input = input.ix[:(input.shape[0]-1)]
time_slot_dummies = pd.get_dummies(input['time_slot'])
for i in range(142):
    input['time_slot_'+str(i)] = time_slot_dummies[float(i)]
district_dummies = pd.get_dummies(input['district_id'])
for i in range(65):
    input['district_id_'+str(i+1)] = district_dummies[str(i+1)]
input['fri'] = input['day']//7==1
input['sat'] = input['day']//7==2
input['sun'] = input['day']//7==3
input['mon'] = input['day']//7==4
input['tue'] = input['day']//7==5
input['wed'] = input['day']//7==6
input.drop('district_id', axis=1, inplace=True)
input.drop('time_slot', axis=1, inplace=True)

In [41]:
input

,day,gap,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,...,district_id_62,district_id_63,district_id_64,district_id_65,fri,sat,sun,mon,tue,wed
key,,,,,,,,,,,,,,,,,,,,,
1_1_0,1.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False
1_1_1,1.0,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False
1_1_2,1.0,10.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False
1_1_3,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False
1_1_4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False
1_1_5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False
1_1_6,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False
1_1_7,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False
1_1_8,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,False,False,False,False,False,False


In [42]:
output

,day,time_slot,gap,gap_slot
key,,,,
1_1_1,1.0,1.0,7.0,1.0
1_1_2,1.0,2.0,10.0,2.0
1_1_3,1.0,3.0,5.0,1.0
1_1_4,1.0,4.0,1.0,0.0
1_1_5,1.0,5.0,1.0,0.0
1_1_6,1.0,6.0,6.0,1.0
1_1_7,1.0,7.0,2.0,0.0
1_1_8,1.0,8.0,6.0,1.0
1_1_9,1.0,9.0,6.0,1.0


In [52]:
# Convert the data we need into np array for training network
import numpy as np
train_x = input.as_matrix().astype(float)
train_y = output['gap'].as_matrix().astype(float)
train_y_class = output['gap_slot'].as_matrix().astype(float)
train_y_class1 = output['class'].as_matrix().astype(float)

In [ ]:
train_y_diff = train_y.copy()
for i in range(train_y_diff.shape[0]-1):
    train_y_diff[i] = train_y_diff[i+1] - train_y_diff[i]

In [ ]:
res

### Export the np array and dataframe as txt for future convenience

In [ ]:
print(train_x.shape, train_y.shape, train_y_class.shape)

In [53]:
np.savetxt('train_x.txt', train_x, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ')
np.savetxt('train_y.txt', train_y, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ')
np.savetxt('train_y_class.txt', train_y_class, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ')
# np.savetxt('train_y_diff.txt', train_y_diff, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ')
np.savetxt('train_y_class1.txt', train_y_class1, fmt='%.18e', delimiter=' ', newline='\n', header='', footer='', comments='# ')

In [ ]:
res.to_csv('res.txt', sep = ' ')
input.to_csv('input.txt', sep=' ')
output.to_csv('output.txt', sep=' ')

### Direct read the modified dataframe from csv

In [ ]:
import pandas as pd

res = pd.read_csv('res.txt',  delim_whitespace = 1, ) 
input = pd.read_csv('input.txt',  delim_whitespace = 1,) 
output = pd.read_csv('output.txt',  delim_whitespace = 1,) 

In [ ]:
input.shape

In [ ]:
input = pd.DataFrame(res[['day', 'district_id', 'gap', 'time_slot']])
output = pd.DataFrame(res[['day', 'time_slot', 'gap', 'gap_slot']])

input = input.ix[1:]
output = output.ix[:(output.shape[0]-2)]

In [ ]:
input.drop('key', axis=1, inplace=True)
output.drop('key', axis=1, inplace=True)

In [ ]:
output

In [ ]:
train_y = output['gap'].as_matrix().astype(float)
train_y_diff = train_y.copy()
for i in range(train_y_diff.shape[0]-1):
    train_y_diff[i] = train_y_diff[i+1] - train_y_diff[i]

In [43]:
input = input.reset_index()

In [44]:
output.loc[(output['gap']==0),'class'] = 0
output.loc[(output['gap']==1),'class'] = 1
output.loc[(output['gap']==2),'class'] = 2
output.loc[(output['gap']==3),'class'] = 3
output.loc[(output['gap']==4),'class'] = 4
output.loc[(output['gap']>4),'class'] = 5

In [45]:
for i in range(input.shape[0]-2):
    if i%5000==0:
        print(i)
    input.loc[i+1, 'gap_past1'] = input.loc[i, 'gap']
    input.loc[i+2, 'gap_past2'] = input.loc[i, 'gap']

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000


In [46]:
np.sum(train_y==1)

31821

In [50]:
output

,day,time_slot,gap,gap_slot,class
key,,,,,
1_1_1,1.0,1.0,7.0,1.0,5.0
1_1_2,1.0,2.0,10.0,2.0,5.0
1_1_3,1.0,3.0,5.0,1.0,5.0
1_1_4,1.0,4.0,1.0,0.0,1.0
1_1_5,1.0,5.0,1.0,0.0,1.0
1_1_6,1.0,6.0,6.0,1.0,5.0
1_1_7,1.0,7.0,2.0,0.0,2.0
1_1_8,1.0,8.0,6.0,1.0,5.0
1_1_9,1.0,9.0,6.0,1.0,5.0


In [47]:
input.loc[199582, 'gap_past1'] = 1.0
input.loc[0, 'gap_past1'] = 0
input.loc[0, 'gap_past2'] = 0
input.loc[1, 'gap_past2'] = 0

In [48]:
input = input.drop('key', 1)
input = input.drop('day', 1)

In [49]:
input

,gap,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,district_id_64,district_id_65,fri,sat,sun,mon,tue,wed,gap_past1,gap_past2
0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,False,False,False,False,False,False,0.0,0.0
1,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,False,False,False,False,False,False,9.0,0.0
2,10.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,False,False,False,False,False,False,7.0,9.0
3,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,False,False,False,False,False,False,10.0,7.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,False,False,False,False,False,False,5.0,10.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,False,False,False,False,False,False,1.0,5.0
6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,False,False,False,False,False,False,1.0,1.0
7,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,False,False,False,False,False,False,6.0,1.0
8,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,False,False,False,False,False,False,2.0,6.0
9,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,False,False,False,False,False,False,6.0,2.0


In [ ]:
list(input.columns.values)

In [ ]:
input

In [35]:
output

,day,time_slot,gap,gap_slot,L1,L2,L3,L4,class
key,,,,,,,,,
1_1_1,1.0,1.0,7.0,1.0,1399,318,102,94,5.0
1_1_2,1.0,2.0,10.0,2.0,1491,322,99,64,5.0
1_1_3,1.0,3.0,5.0,1.0,1490,287,98,78,5.0
1_1_4,1.0,4.0,1.0,0.0,1425,302,95,51,1.0
1_1_5,1.0,5.0,1.0,0.0,1327,313,94,66,1.0
1_1_6,1.0,6.0,6.0,1.0,1361,258,68,55,5.0
1_1_7,1.0,7.0,2.0,0.0,1395,280,97,69,2.0
1_1_8,1.0,8.0,6.0,1.0,1348,272,93,97,5.0
1_1_9,1.0,9.0,6.0,1.0,1417,236,93,53,5.0
